# **Fine-Tune W2V2-Bert for low-resource ASR with 🤗 Transformers**

[Fine-tuning XLS-R on Multi-Lingual ASR](https://huggingface.co/blog/fine-tune-xlsr-wav2vec2) and [Fine-tuning MMS Adapter Models for Multi-Lingual ASR](https://huggingface.co/blog/mms_adapters).

## Notebook Setup

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri May 31 08:02:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%capture
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install jiwer
!pip install accelerate -U

In [ ]:
import IPython.display as ipd
import numpy as np
import random
from datasets import ClassLabel
import pandas as pd
from IPython.display import display, HTML
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
%%capture
!apt install git-lfs

## Prepare Data, Tokenizer, Feature Extractor

### Create `Wav2Vec2CTCTokenizer`

In [ ]:
from datasets import load_dataset, load_metric, Audio

In [ ]:
dataset = load_dataset("tbkazakova/even_speech", split='train').train_test_split(test_size=0.2)

Resolving data files:   0%|          | 0/1774 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1773 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 1418
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 355
    })
})

In [ ]:
even_train = dataset['train']
even_test = dataset['test']

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(even_train, num_examples=5)

,audio,transcription
0,"{'path': '/root/.cache/huggingface/datasets/downloads/fbcfed7d2a2b6f0524cc7a950c22fc384d4c173a69184d89eea4e213b1323cf7', 'array': [-0.0035400390625, -0.002777099609375, -0.001373291015625, -0.000274658203125, 0.000457763671875, 0.001434326171875, 0.002532958984375, 0.003753662109375, 0.004058837890625, 0.0045166015625, 0.00494384765625, 0.00482177734375, 0.004638671875, 0.00421142578125, 0.0035400390625, 0.00250244140625, 0.000885009765625, -0.000518798828125, -0.001312255859375, -0.00250244140625, -0.00372314453125, -0.00469970703125, -0.00531005859375, -0.00604248046875, -0.006622314453125, -0.006195068359375, -0.005340576171875, -0.003936767578125, -0.0028076171875, -0.00128173828125, 0.0008544921875, 0.002410888671875, 0.004150390625, 0.005950927734375, 0.007049560546875, 0.00726318359375, 0.0069580078125, 0.00628662109375, 0.005523681640625, 0.004486083984375, 0.003265380859375, 0.0020751953125, 0.000518798828125, -0.000762939453125, -0.001190185546875, -0.001251220703125, -0.00152587890625, -0.002197265625, -0.0035400390625, -0.004638671875, -0.00543212890625, -0.005828857421875, -0.0048828125, -0.003753662109375, -0.002532958984375, -0.0006103515625, 0.002044677734375, 0.0047607421875, 0.00634765625, 0.00701904296875, 0.006439208984375, 0.004608154296875, 0.0020751953125, 0.0, -0.001190185546875, -0.001678466796875, -0.001495361328125, -0.001373291015625, -0.001068115234375, -0.0010986328125, -0.00146484375, -0.001861572265625, -0.002532958984375, -0.00408935546875, -0.00592041015625, -0.00677490234375, -0.006561279296875, -0.00494384765625, -0.002349853515625, 0.00054931640625, 0.002777099609375, 0.00384521484375, 0.003814697265625, 0.003173828125, 0.00244140625, 0.0008544921875, -0.000579833984375, -0.00189208984375, -0.002899169921875, -0.003173828125, -0.0028076171875, -0.00201416015625, -0.001800537109375, -0.002197265625, -0.003326416015625, -0.004425048828125, -0.00494384765625, -0.00482177734375, -0.003753662109375, -0.001861572265625, ...], 'sampling_rate': 22050}",Хөвки төррэ мөв тилкамкандаӈан дюгулин
1,"{'path': '/root/.cache/huggingface/datasets/downloads/3c980ae5bcc7ff73e9817ebccc6567da59570b4dd1ee9fa1747cf9c04b13407e', 'array': [0.0001220703125, 9.1552734375e-05, -0.000213623046875, -0.00018310546875, -3.0517578125e-05, -0.000152587890625, 3.0517578125e-05, 0.00018310546875, 0.000244140625, 3.0517578125e-05, 3.0517578125e-05, 3.0517578125e-05, -0.000213623046875, 6.103515625e-05, 0.000213623046875, 0.0001220703125, -6.103515625e-05, 6.103515625e-05, 0.0003662109375, 0.000152587890625, -0.000152587890625, -0.00018310546875, -0.000244140625, -0.000335693359375, -0.000213623046875, -0.000213623046875, -0.00018310546875, -6.103515625e-05, 0.0001220703125, 0.0001220703125, -6.103515625e-05, 0.0, -6.103515625e-05, -6.103515625e-05, 0.00018310546875, 0.0001220703125, 9.1552734375e-05, 0.0001220703125, 0.000152587890625, 0.000244140625, 0.00018310546875, 0.00018310546875, 0.0, -0.000152587890625, -3.0517578125e-05, 3.0517578125e-05, 6.103515625e-05, 3.0517578125e-05, -6.103515625e-05, 0.0, 6.103515625e-05, -0.000152587890625, -0.00018310546875, 0.0001220703125, -3.0517578125e-05, -0.00018310546875, -0.0001220703125, 0.0, 3.0517578125e-05, 3.0517578125e-05, 6.103515625e-05, -3.0517578125e-05, 6.103515625e-05, 0.000152587890625, 9.1552734375e-05, -9.1552734375e-05, -6.103515625e-05, 3.0517578125e-05, 0.0, 0.000244140625, 0.000244140625, 0.0001220703125, 9.1552734375e-05, -3.0517578125e-05, 6.103515625e-05, 0.00018310546875, 0.000152587890625, 9.1552734375e-05, 9.1552734375e-05, 0.0, 6.103515625e-05, 0.0, -0.0001220703125, -3.0517578125e-05, -6.103515625e-05, 0.0, -3.0517578125e-05, -3.0517578125e-05, 3.0517578125e-05, -6.103515625e-05, 6.103515625e-05, 3.0517578125e-05, -0.00018310546875, -0.000213623046875, -0.0001220703125, 6.103515625e-05, -0.00018310546875, -0.000274658203125, -3.0517578125e-05, -3.0517578125e-05, ...], 'sampling_rate': 22050}",Тадук Авраамдула Хөвки төрэнни ихөнчэ:
2,"{

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

Let's simply remove all characters that don't contribute to the meaning of a word and cannot really be represented by an acoustic sound and normalize the text.

In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\»\«\„\‐\–]'

def remove_special_characters(batch):
    # remove special characters
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()

    return batch

In [ ]:
even_train = even_train.map(remove_special_characters)
even_test = even_test.map(remove_special_characters)

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Let's look at the processed text labels again.

In [ ]:
show_random_elements(even_train.remove_columns(["audio"]))

,transcription
0,ноӈортон коллидюр хөбөлчэл
1,хину хиргэччим
2,няниндула бихи дэгилбу ичэлдэ
3,хинимкилдук бугин эхэлдэ
4,көкэмидэ урдин
5,эсимэсукэн хиладдытки бэйтэки самариянкан дагамрин
6,тар эчин бихин
7,тала хөвки төрэмутнэ улкуча тиэми тар городу вавилон гөникэн гөрбучэл
8,тадук ноӈортон эрэгэр бэгэнтэдир
9,муттэ амандулат


In CTC, it is common to classify speech chunks into letters, so we will do the same here.
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars.
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = even_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=even_train.column_names)
vocab_test = even_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=even_test.column_names)

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'а': 1,
 'б': 2,
 'в': 3,
 'г': 4,
 'д': 5,
 'е': 6,
 'з': 7,
 'и': 8,
 'й': 9,
 'к': 10,
 'л': 11,
 'м': 12,
 'н': 13,
 'о': 14,
 'п': 15,
 'р': 16,
 'с': 17,
 'т': 18,
 'у': 19,
 'ф': 20,
 'х': 21,
 'ц': 22,
 'ч': 23,
 'ш': 24,
 'ы': 25,
 'ь': 26,
 'э': 27,
 'ю': 28,
 'я': 29,
 'ё': 30,
 'ӈ': 31,
 'ө': 32,
 'ӫ': 33}

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in training set.

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

Finally, we also add a padding token that corresponds to CTC's "*blank token*".


The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

36

Cool, now our vocabulary is complete and consists of 36 tokens, which means that the linear layer that we will add on top of the pretrained W2V-BERT checkpoint will have an output dimension of 36.

Let's now save the vocabulary as a json file.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to load the vocabulary into an instance of the `Wav2Vec2CTCTokenizer` class.

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec-bert-2.0-even-biblical"

and upload the tokenizer to the [🤗 Hub](https://huggingface.co/)

In [ ]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/tbkazakova/w2v-bert-2.0-even-colab-CV16.0/commit/7290507d09f58cf27b3199522a4cfcab567830ea', commit_message='Upload tokenizer', commit_description='', oid='7290507d09f58cf27b3199522a4cfcab567830ea', pr_url=None, pr_revision=None, pr_num=None)

### Create `SeamlessM4TFeatureExtractor`

The role of the `SeamlessM4TFeatureExtractor` is to prepare the raw audio input in a format that the model can "understand". It therefore maps the sequence of one-dimensional amplitude values (aka the raw audio input) to a two-dimensional matrix of log-mel spectrogram values. The latter encodes the signal frequency information as a function of time. See [this section](https://huggingface.co/learn/audio-course/chapter1/audio_data#the-frequency-spectrum) from the Audio Transformers course to learn more about spectrograms and why they are important.

Unlike the tokenizer, the feature extractor doesn't need to be "learned" from the data, so we can load it directly from the [initial model checkpoint](https://huggingface.co/facebook/w2v-bert-2.0).


In [ ]:
from transformers import SeamlessM4TFeatureExtractor

feature_extractor = SeamlessM4TFeatureExtractor(feature_size=80, num_mel_bins=80, sampling_rate=16000, padding_value=0.0)

Great, W2V-BERT's feature extraction pipeline is thereby fully defined!

For improved user-friendliness, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2BertProcessor` class so that one only needs a `model` and `processor` object.

In [ ]:
from transformers import Wav2Vec2BertProcessor

processor = Wav2Vec2BertProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.push_to_hub(repo_name)

Next, we can prepare the dataset.

### Preprocess Data


In [ ]:
even_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/29efc52131f2a4f6cbddaa641a21410181863e6bd5bf39096ba46a582a7ec0bc',
 'array': array([-6.10351562e-05,  9.15527344e-05,  0.00000000e+00, ...,
         3.17382812e-03,  1.67846680e-03,  9.15527344e-05]),
 'sampling_rate': 22050}

W2V-BERT expects the input in the format of a 1-dimensional array of 16 kHz. This means that the audio file has to be loaded and resampled.

 Thankfully, `datasets` does this automatically by calling the other column `audio`. Let try it out.

In [ ]:
even_train = even_train.cast_column("audio", Audio(sampling_rate=16_000))
even_test = even_test.cast_column("audio", Audio(sampling_rate=16_000))

Let's take a look at `"audio"` again.

In [ ]:
even_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/29efc52131f2a4f6cbddaa641a21410181863e6bd5bf39096ba46a582a7ec0bc',
 'array': array([-1.21104122e-05,  3.83910628e-05,  1.51483182e-04, ...,
         3.50215565e-03,  1.44801906e-03,  0.00000000e+00]),
 'sampling_rate': 16000}

This seemed to have worked! Let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded.

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [ ]:
rand_int = random.randint(0, len(even_train)-1)

print(even_train[rand_int]["transcription"])
ipd.Audio(data=even_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

тар хотарандули иисус иэлтэннэн бичэ


It seems like the data is now correctly loaded and resampled.

It can be heard, that the speakers change along with their speaking rate, accent, and background environment, etc. Overall, the recordings sound acceptably clear though, which is to be expected from a crowd-sourced read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [ ]:
rand_int = random.randint(0, len(even_train)-1)

print("Target text:", even_train[rand_int]["transcription"])
print("Input array shape:", even_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", even_train[rand_int]["audio"]["sampling_rate"])

Target text: нян хагахрит кунича
Input array shape: (24176,)
Sampling rate: 16000


Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Finally, we can leverage `Wav2Vec2BertProcessor` to process the data to the format expected by `Wav2Vec2BertForCTC` for training. To do so let's make use of Dataset's [`map(...)`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=map#datasets.DatasetDict.map) function.

First, we load and resample the audio data, simply by calling `batch["audio"]`.
Second, we extract the `input_features` from the loaded audio file. In our case, the `Wav2Vec2BertProcessor` creates a more complex representation as the raw waveform, known as [Log-Mel feature extraction](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum).
Third, we encode the transcriptions to label ids.


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["input_length"] = len(batch["input_features"])

    batch["labels"] = processor(text=batch["transcription"]).input_ids
    return batch

Let's apply the data preparation function to all examples.

In [ ]:
even_train = even_train.map(prepare_dataset, remove_columns=even_train.column_names)
even_test = even_test.map(prepare_dataset, remove_columns=even_test.column_names)

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

**Note**: `datasets` automatically takes care of audio loading and resampling. If you wish to implement your own costumized data loading/sampling, feel free to just make use of the `"path"` column instead and disregard the `"audio"` column.

Awesome, now we are ready to start training!

## Training

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/7e61d56a45c19284cfda0cee8995fb552f6b1f4e/examples/pytorch/speech-recognition/run_speech_recognition_ctc.py#L219).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_features` and `labels` differently and thus applies to separate padding functions on them. This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2BertProcessor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )
        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-8-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
cer_metric = load_metric("cer")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for cer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/cer/cer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)


    return {"wer": wer, "cer": cer}

Now, we can load the pretrained checkpoint of [Wav2Vec2-XLS-R-300M](https://huggingface.co/facebook/wav2vec2-xls-r-300m). The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2BertForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

Since, we're only training a small subset of weights, the model is not prone to overfitting. Therefore, we make sure to disable all dropout layers.

In [ ]:
from transformers import Wav2Vec2BertForCTC

model = Wav2Vec2BertForCTC.from_pretrained(
    "facebook/w2v-bert-2.0",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.0,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    add_adapter=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

Some weights of Wav2Vec2BertForCTC were not initialized from the model checkpoint at facebook/w2v-bert-2.0 and are newly initialized: ['adapter.layers.0.ffn.intermediate_dense.bias', 'adapter.layers.0.ffn.intermediate_dense.weight', 'adapter.layers.0.ffn.output_dense.bias', 'adapter.layers.0.ffn.output_dense.weight', 'adapter.layers.0.ffn_layer_norm.bias', 'adapter.layers.0.ffn_layer_norm.weight', 'adapter.layers.0.residual_conv.bias', 'adapter.layers.0.residual_conv.weight', 'adapter.layers.0.residual_layer_norm.bias', 'adapter.layers.0.residual_layer_norm.weight', 'adapter.layers.0.self_attn.linear_k.bias', 'adapter.layers.0.self_attn.linear_k.weight', 'adapter.layers.0.self_attn.linear_out.bias', 'adapter.layers.0.self_attn.linear_out.weight', 'adapter.layers.0.self_attn.linear_q.bias', 'adapter.layers.0.self_attn.linear_q.weight', 'adapter.layers.0.self_attn.linear_v.bias', 'adapter.layers.0.self_attn.linear_v.weight', 'adapter.layers.0.self_attn_conv.bias', 'adapter.layers.0.self_

In a final step, we define all parameters related to training.
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` was heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=300,
  eval_steps=200,
  logging_steps=200,
  learning_rate=5e-5,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=even_train,
    eval_dataset=even_test,
    tokenizer=processor.feature_extractor,
)



---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

Depending on what GPU was allocated to your google colab it might be possible that you are seeing an `"out-of-memory"` error here. In this case, it's probably best to reduce `per_device_train_batch_size` to 8 or even less and increase [`gradient_accumulation`](https://huggingface.co/transformers/master/main_classes/trainer.html#trainingarguments).

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
300,2.046500,inf,0.769182


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


TrainOutput(global_step=440, training_loss=1.739978686246005, metrics={'train_runtime': 1126.9185, 'train_samples_per_second': 12.583, 'train_steps_per_second': 0.39, 'total_flos': 9.970586122667789e+17, 'train_loss': 1.739978686246005, 'epoch': 9.887640449438202})

You can now upload the result of the training to the 🤗 Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

events.out.tfevents.1717188729.6f6b557b0f8a.129.0:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tbkazakova/wav2vec-bert-2.0-even/commit/1589f8045572233f5ac038727901d0adf8efadb3', commit_message='End of training', commit_description='', oid='1589f8045572233f5ac038727901d0adf8efadb3', pr_url=None, pr_revision=None, pr_num=None)

For more examples of how W2V-BERT can be fine-tuned, please take a look at the [official speech recognition examples](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#examples).

### Evaluation

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2BertProcessor

model = AutoModelForCTC.from_pretrained("tbkazakova/wav2vec-bert-2.0-even").to("cuda")
processor = Wav2Vec2BertProcessor.from_pretrained("tbkazakova/wav2vec-bert-2.0-even")

Let's process the audio, run a forward pass and predict the ids

In [ ]:
input_dict = even_test[0]

logits = model(torch.tensor(input_dict["input_features"]).to("cuda").unsqueeze(0)).logits

pred_ids = torch.argmax(logits, dim=-1)[0]


Finally, we can decode the example from the predicted tokens:

In [ ]:
processor.decode(pred_ids)

'авраам халалбигэрридиттэкэн охукан илан б илгэмача'

And compare it to the reference transcription:

In [ ]:
processor.decode(input_dict["labels"]).lower()

'аврам яхалалби өгөриди итэкэн ахукан илан бөӫл илгамачал'

In [ ]:
len(even_test)

355

In [ ]:
predictions = []
references = []
for i in range(len(even_test)):
    input_dict = even_test[i]
    logits = model(torch.tensor(input_dict["input_features"]).to("cuda").unsqueeze(0)).logits
    pred_ids = torch.argmax(logits, dim=-1)[0]
    predictions.append(processor.decode(pred_ids))
    references.append(processor.decode(input_dict["labels"]).lower())


wer = wer_metric.compute(predictions=predictions, references=references)
cer = cer_metric.compute(predictions=predictions, references=references)

In [ ]:
wer, cer

(0.7643186306780777, 0.20378542708639796)

In [ ]:
pred_real = pd.DataFrame()
pred_real['pred'] = predictions
pred_real['real'] = references
pred_real

,pred,real
0,авраам халалбигэрридиттэкэн охукан илан б илгэ...,аврам яхалалби өгөриди итэкэн ахукан илан бөӫл...
1,иисус ноӈонтикин гөнин,исус ноӈонтикин гөнин
2,тачинда ивитнэкрин,тачинта левит некрин
3,эмнэкэн исус варисий нянамам аавамӈанэӈнэмин,өмнэкэн исус фарисей нян тамам тавамӈа нэӈнэми
4,тарит нойхонтэ дэгивтинчэ,тарит ной хөнтэ дэгив тинчэ
...,...,...
350,идэкмуткилибугэди,идук мут килепу гадип
351,хөвки гөнчэн,хөвки гөнчэ
352,төрду одо ӈалби булбунян далгэнкэлбу ачалтаде,төрдук одаӈалби бөӫлбу нян дэлгэнкэлбу ачалтадим
353,тар хакита даиан эмниди,тар хиакита далилан эмниди
